# Trouver les thèmes des livres disponibles sur Gutenberg

La deuxième étape de l'extraction des données consiste à trouver les thèmes des livres dont on sait qu'on dispose des textes dans Gutenberg. Cependant, comme Gutenberg ne fournit pas cette information, nous avons utilisé l'API **OpenLibrary**, qui est l'une des rares API regroupant les thèmes des livres. 

## Sélection des auteurs
Pour optimiser l'extraction, nous avons défini une liste d'auteurs à partir des pages Wikipedia recensant les romanciers français, en particulier ceux mentionnés dans la [catégorie des romanciers français par siècle](https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:Romancier_fran%C3%A7ais_par_si%C3%A8cle). Nous avons croisé cette liste avec nos connaissances en prépa B/L pour garder uniquement les auteurs dits "classiques". Cette approche présente trois avantages :
1. **Maximiser les chances de trouver les thèmes des livres** : Les auteurs classiques ont écrit des ouvrages qui sont souvent cités et reconnus, augmentant ainsi les probabilités d’obtenir des informations sur leurs thèmes.
2. **Réduire la taille de la base de données** : En limitant la sélection aux auteurs classiques, nous évitons une requête trop large qui pourrait être difficile à traiter en termes de temps et qui aurait un impact environnemental plus fort.
3. **Faciliter l'application du projet** : En se concentrant sur des auteurs classiques, les livres sont non seulement largement accessibles, mais également facilement retrouvables dans la vie réelle pour des applications pratiques.


## Nettoyage des données brutes
Nous partons de la liste des livres (fichier `livres_fr_triés.csv`) pour lesquels nous savons que les textes complets sont disponibles. Après avoir récupéré ces données brutes, nous effectuons un nettoyage pour obtenir un dataframe regroupant les noms des auteurs et les titres des livres. 

## Importations des modules

In [46]:
import pandas as pd
import requests
import time
from urllib.parse import quote
from langdetect import detect
from fuzzywuzzy import fuzz 
import s3fs


In [47]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

MY_BUCKET = "arnaudbrrt"
fs.ls(MY_BUCKET) 



['arnaudbrrt/Data_libroguessr', 'arnaudbrrt/diffusion']

In [49]:
#TODO supprimer 
"""FILE_PATH_IN_S3 = f"{MY_BUCKET}/Data_libroguessr/livres_fr_triés.csv"

with fs.open(FILE_PATH_IN_S3, "r") as file_in:
    data = pd.read_csv(file_in)"""
data = pd.read_csv('livres_fr_triés.csv')
data

,Description,Index
0,"La Comédie humaine, Vol. 17, Études de mœurs, ...",74126
1,Œuvres complètes de Guy de Maupassant - volume...,73633
2,"La Comédie humaine, Volume 16, Études philosop...",73552
3,"Œuvres complètes de Victor Hugo, by Victor Hug...",72885
4,"La Comédie humaine, Volume XV, Études philosop...",72034
...,...,...
471,"Le Tour du Monde en 80 Jours, by Jules Verne ...",800
472,"De La Terre a La Lune, by Jules Verne ...",799
473,"Le Rouge et le Noir, by Stendhal ...",798
474,"L'Abbesse de Castro, by Stendhal ...",797


## Liste des fonctions 

In [50]:
def f_get_themes(author, title):
    """
    This function fetches information about a book by a given author from the Open Library API.
    It returns the themes (subjects) associated with the book that has the largest number of subjects among the results.

    Arguments:
        - author (str): The name of the book's author.
        - title (str): The title of the book.

    Workflow:
        1. Constructs a query to the Open Library API using the author's name and book's title.
        2. Retrieves JSON results containing information about books matching the query.
        3. Filters books to keep only those that have defined themes ("subject").
        4. Selects the book with the largest number of themes from the filtered results.
        5. Returns the selected book's information: author, title, year of publication, and themes.

    Returns:
        - A list containing a single tuple in the format:
          [(author, title, year, themes)]
          where:
            - author (str): The author's name.
            - title (str): The book's title.
            - year (str): The book's first publication year (if available).
            - themes (str): The book's associated themes, joined into a string separated by commas.
          If no book with themes is found, it returns a list with empty values:
          [(author, title, '', '')].
    """
    
    url_api = "https://openlibrary.org/search.json?"

    # Build the API URL with query parameters for author and title
    url = url_api + f'q=author:{author}&fields={title},first_publish_year,subject'
    req = requests.get(url)  # Send the HTTP request
    time.sleep(2)  # Pause for 2 seconds to avoid overloading the API
    
    # Check if the request was successful (HTTP status code 200)
    if req.status_code == 200:
        books = req.json().get("docs", [])  # Retrieve the list of books from the JSON response
        
        # Filter books that contain the "subject" field (themes)
        books_with_themes = [
            book for book in books
            if book.get("subject")  # Keep only books with defined themes
        ]
        
        if books_with_themes:  # If books with themes are found
            # Select the book with the largest number of themes
            best_book = max(books_with_themes, key=lambda book: len(book["subject"]))
            
            # Retrieve the themes of the selected book and join them into a single string
            themes = ", ".join(best_book.get("subject", ''))  
            
            # Retrieve the year of first publication
            year = best_book.get("first_publish_year", '')  
            
            # Return the book information as a list of a single tuple
            return [(author, title, year, themes)] 
        else:
            # If no book with themes is found, return empty values
            return [(author, title, '', '')]
    else:
        # If the request fails, return empty values
        return [(author, title, '', '')]

def f_transform_themes_into_df(authors, titles):
    """
    This function takes two lists as input: a list of authors and a list of titles.
    It calls the `f_get_themes` function for each author-title combination and returns a flattened list of tuples containing the book information.

    Arguments:
        - authors (list): List of authors' names.
        - titles (list): List of book titles.

    Returns:
        - A DataFrame containing columns: 'Author', 'Title', 'Year', 'Themes' with the book information for each author-title combination.
    """
    list_books = []

    # For each author and title, retrieve the book information
    for author, title in zip(authors, titles):
        list_books.append(f_get_themes(author, title))

    # Flatten the list of lists of tuples into a single list
    flattened_books = [book for author_data in list_books for book in author_data]

    return pd.DataFrame(flattened_books, columns=['Author', 'Title', 'Year', 'Themes'])


def f_remove_similar_books_from_df(df, threshold=80):
    """
    Removes rows with similar book titles from a DataFrame, grouped by the same author. 
    Titles are compared using a similarity threshold.

    Parameters:
    -----------
    df : pandas.DataFrame
        A DataFrame containing book data. It must have at least two columns:
        - 'Author': The name of the author of the book.
        - 'Title': The title of the book.
    threshold : int, optional (default=80)
        The minimum similarity percentage (0-100) between two titles for them to be considered duplicates.

    Returns:
    --------
    pandas.DataFrame
        A cleaned DataFrame with duplicate titles (based on similarity) removed. 
        The index is reset after removing the rows.
    """

    # Initialize a list to store the indices of rows to be removed.
    to_remove = []

    # Group books by the 'Author' column and get the list of titles for each author.
    list_books_per_author = df.groupby('Author')['Title'].apply(list)

    # Iterate through each group of books by the same author.
    for author, books in list_books_per_author.items():
        # Compare each title with all subsequent titles in the list.
        for i in range(len(books)):
            for j in range(i + 1, len(books)):
                # Compute the similarity ratio between two titles using fuzzy matching.
                similarity = fuzz.ratio(books[i], books[j])
                
                # If the similarity exceeds the threshold, mark the second title for removal.
                if similarity >= threshold:
                    # Find the index of the duplicate title in the original DataFrame.
                    index_to_remove = df[(df['Author'] == author) & (df['Title'] == books[j])].index
                    # Append the index to the removal list.
                    to_remove.extend(index_to_remove)

    # Drop all rows with indices marked for removal and reset the index of the resulting DataFrame.
    df = df.drop(to_remove).reset_index(drop=True)
    
    return df



def f_map_themes(row):
    """
    Maps individual themes from a comma-separated input string to their corresponding categories based on 
    the `themes_mapping` dictionary. The function splits the input string of themes, searches for each theme 
    in the dictionary, and returns a string of category names.

    Parameters:
    row (str): A string of themes, separated by commas and spaces. Each theme corresponds to one or more 
               categories in the `themes_mapping` dictionary.

    Returns:
    str: A comma-separated string containing the categories (keys) from the `themes_mapping` dictionary 
         that correspond to the input themes. If a theme is found in multiple categories, only the first 
         matching category is returned for that theme.

    """
    
    # Split the input string by comma and space (", ") into individual themes
    themes = row.split(", ")
    
    # Initialize an empty list to store the mapped themes (categories)
    mapped_themes = []  
    
    # Iterate through each theme in the list
    for theme in themes:
        # Loop through each key-value pair in the themes_mapping dictionary
        for key, values in themes_mapping.items():
            # If the theme is found in the values (list) for the current key
            if theme in values:
                # Add the corresponding category (key) to the list of mapped themes
                mapped_themes.append(key)  
                break  # Exit the inner loop once a match is found
    
    # Return the mapped themes as a comma-separated string
    return ", ".join(mapped_themes)

## Nettoyage du DataFrame des livres pour requêter _in fine_ l'API OpenLibrary 

 >**Remarque** : Cette partie du script nécessite un temps d'exécution de plus de 30 minutes.

In [51]:
# Séparation de la colonne en deux
data[['Title', 'Author']] = data['Description'].str.split(' by ', expand=True)

# Suppression de l'ancienne
df_books = data.drop(columns=['Description'])

# Inversion de l'ordre des colonnes
df_books = df_books[['Author', 'Title']]
df_books.columns = ['Author', 'Title']

# Pour les livres ayant deux auteurs, on remplace le "and" par une virgule
df_books['Author'] = df_books['Author'].str.replace(' and ', ', ') 

# On nettoie le nom des auteurs, en enlevant les informations suivant leur nom
df_books['Author'] = df_books['Author'].str.split('\n').str[0]

# On retire les livres qui n'ont pas d'auteur associé
df_books = df_books[df_books['Author'].notna()]
df_books.reset_index(drop=True, inplace=True)

# On supprime les numéros de volume des livres losqu'il est donné
df_books['Title'] = df_books['Title'].str.replace(r'\s*,?\s*(Tome|tome|Volume|volume|Vol.|vol.|V)\s+\w+', '', regex=True)
df_books['Title'] = df_books['Title'].str.rstrip(' -') # Enlever le tirer en trop apparaissant en fin de ligne

# On retire les sous-titres des livres
df_books['Title'] = df_books['Title'].str.replace(r':.*', '', regex=True)

# On enlève les virgules inutiles en fin de ligne
df_books['Title'] = df_books['Title'].str.rstrip(',')

# On enlève les tirets inutiles en fin de ligne
df_books['Title'] = df_books['Title'].str.rstrip('-').str.rstrip()

# Après ce nettoyage, on remarque que des livres ont des titres similaires. De ce fait, on les nettoie
df_books = f_remove_similar_books_from_df(df_books)

# On récupère les thèmes des livres 
df_books = f_transform_themes_into_df(df_books['Author'], df_books['Title'])

# On retire les lignes sans thèmes
df_books = df_books[df_books['Themes'] != '']
df_books.reset_index(drop=True, inplace=True)

df_books

KeyboardInterrupt: 

# Traduction via l'API Lingva Translate

## Pourquoi utiliser cette API pour traduire les thèmes ?
Dans ce projet, nous avons choisi d'utiliser l'API **Lingva Translate** pour traduire les thèmes. Voici les principales raisons qui motivent cette décision :

### Gratuité :
L'API Lingva est entièrement gratuite, ce qui en fait un choix économique, particulièrement adapté aux projets exploratoires ou personnels.

### Pas de token requis :
Contrairement à de nombreuses autres API de traduction, Lingva ne nécessite pas de clé API ou de processus d'inscription complexe. Cela simplifie considérablement l'intégration dans nos scripts ou notebooks.

 >**Remarque** : Cette partie du script nécessite un temps d'exécution de plus de 10 minutes.

In [29]:
url = "https://lingva.ml/api/v1/"
source = 'auto/'
dest = 'fr/'

for i in range(len(df_books['Themes'])):
    if detect(df_books['Themes'][i]) != 'fr' :
        url_api_transl = url + source + dest + df_books['Themes'][i]
        response = requests.get(url_api_transl)
        if response.status_code == 200:
            translated_themes = response.json().get("translation", "")
            df_books.loc[i,'Themes'] = translated_themes
        time.sleep(2)

# Nettoyage et regroupement des thèmes

Une fois que les thèmes des livres ont été traduits, nous procédons à une phase de nettoyage et de regroupement en thèmes plus généraux. 
Voici les étapes principales de cette démarche :

1. **Nettoyage des thèmes non pertinents** :  
   Certains thèmes traduits peuvent être trop spécifiques ou ne pas avoir d'intérêt dans le contexte du projet. Ces thèmes sont supprimés, ce qui peut entraîner l'absence de thèmes associés à certains livres.

2. **Regroupement des thèmes dans des catégories générales** :  
   Les thèmes pertinents sont regroupés manuellement dans des catégories plus larges et significatives grâce à un processus de *mapping*. Chaque catégorie regroupe une liste de mots-clés qui lui sont associés, ce qui permet de simplifier l'analyse.

3. **Création manuelle du *mapping*** :  
   Le *mapping* est conçu manuellement pour s'assurer que chaque thème est attribué à la catégorie appropriée. Ce travail manuel garantit une classification cohérente et alignée avec les objectifs du projet.

In [30]:
themes_mapping = {
    "Ambitions": ["Ambitions", "Luxe", "Pouvoir", "Richesse"],
    "Antiquité": ["Antiquités", "Empire romain"],
    "Argent": ["Argent", "Finance", "Luxe", "Richesse"],
    "Arts et Culture": ["Illustrations", "Livret", "Musique", "Opéras", "Partitions"],
    "Aventures": ["Animaux", "Découvertes", "Explorations", "Histoires d'aventure", "mer", "naufrages", "Voyages", "Voyages philosophiques", "Voyages imaginaires"],
    "Baroque": ["Émotion", "Exagération", "Illusion", "Mouvement", "Ornement"],
    "Biographie": ["Autobiographie", "Biographie", "Fiction biographique"],
    "Censure": ["Censure", "Critique littéraire", "Interdictions", "Liberté d'expression"],
    "Comédie": ["absurde", "caricature", "comique de situation", "délire", "exagération", "humour", "parodie", "quiproquo", "rire", "satire", "Tromperie"],
    "Critique": ["Critique", "Critique et interprétation", "Histoire de la philosophie", "Histoire politique", "Philosophie", "Satire", "controverse", "Histoire et critique"],
    "Critique d'art": ["Art moderne", "Critique d'art", "Critique d'art", "Peinture", "Arts modernes"],
    "Coutumes": ["Coutumes", "Mœurs et coutumes"],
    "Décadence": ["Chute", "Décadence", "Esthétique", "Fiction"],
    "Drame": ["Drame", "Fiction", "Théâtre"],
    "Éducation": ["Éducation", "Ouvrages de jeunesse jusqu'à 1800"],
    "Éducation et Conduite de la vie": ["Conduite personnelle", "Éducation", "Éducation des filles", "Éducation des princes", "Philosophie de l'éducation"],
    "Épouvante": ["Épouvante", "Horreur", "Suspense", "Thriller"],
    "Esthétique": ["Art", "Esthétique"],
    "Existentialisme": ["Angoisse", "Absurde", "Choix", "Liberté individuelle", "Responsabilité"],
    "Fantastique": ["Fantastique", "Fiction fantastique", "Imaginaire"],
    "Fables": ["Allégories", "Animaux dans la littérature", "Contes moraux", "Fables", "Morales"],
    "Fiction historique": ["Aventures", "Époque", "Événements", "Fiction", "Fiction historique", "Histoire"],
    "Héroïsme": ["Fiction", "Héroïsme"],
    "Histoire": ["Évolutions sociales", "Histoire", "Historique", "Historique et critique", "Politique", "Révolution française", "Révolutions", "Société"],
    "Inégalités": ["Inégalités", "Sociale"],
    "Instincts": ["Comportements", "Instincts"],
    "Intrigues": ["Fiction", "Intrigues"],
    "Justice": ["Droits", "Éthique", "Justice", "Loi"],
    "Liberté": ["Égalité", "Liberté", "Liberté religieuse"],
    "Lumières": ["Égalité", "Liberté", "Philosophie", "Progrès", "Raison", "Éducation"],
    "Médiéval": ["Chevalerie", "Épopée", "Féodalité", "Foi", "Croisades", "Mythologie"],
    "Mélancolie": ["Mélancolie", "Tristesse"],
    "Mœurs": ["Mœurs et coutumes", "Mœurs et usages", "Vie sociale et mœurs"],
    "Mythes et légendes": ["Fables", "Légendes", "Mythes", "Mythologie", "Folklore"],
    "Nature": ["Aspects religieux de la nature", "Histoire naturelle", "La nature dans la littérature", "Nature"],
    "Nature et environnement": ["Conservation", "Environnement", "Écologie", "Nature", "Vie à la campagne"],
    "Oppression": ["Oppression", "Répression"],
    "Passion": ["Amour", "Émotions", "Passion", "Séduction", "érotique", "sexuel", "sexe"],
    "Philosophie": ["Constitution", "Droit", "Égalité", "Existence", "Liberté", "Nationalisme", "Philosophie", "Philosophie du XVIIIe siècle", "Philosophie politique", "Rationalisme", "République", "Vie intellectuelle"],
     "Poésie": ["Ballade", "Chansons", "Épique", "Lyrique", "Poésie", "Sonnet", "Vers"],
    "Postmodernisme": ["Fragmentation", "Intertextualité", "Ironique", "Métafiction", "Pluralisme"],
    "Pouvoir": ["Droit", "Politique et gouvernement", "Pouvoir"],
    "Réalité et illusion": ["Illusion", "Perception", "Réalité", "L'irréel"],
    "Religion": ["Athéisme", "Christianisme", "Dieu", "Église catholique", "Histoire de l'Église", "Religieuses", "Tolérance religieuse", "Affaire Calas"],
    "Récits de guerre et combat": ["Conflits", "Hostilité", "Récits de guerre", "chevaliers", "Histoire militaire"],
    "Renaissance": ["Antiquité", "Art", "Humanisme", "Perspective", "Redécouverte", "Science"],
    "Roman": ["Réalisme", "Roman d'aventure", "Roman d'amour", "Roman philosophique", "Roman historique", "Roman social"],
    "Romantisme": ["amour", "liberté", "mélancolie", "nature", "passion", "sublime"],
    "Satire": ["Caricature", "Critique sociale", "Humour", "Ironie", "Sarcasme", "Satire"],
    "Société et politique": ["Affaire", "Classes sociales", "Commerce", "Conditions sociales", "Culture", "Discours et statut social", "État", "Éducation des princes", "Jurisprudence", "Mœurs", "Nationalisme", "Réformateurs sociaux", "Science politique", "Société", "Socialisme", "Tolérance religieuse"],
    "Surréalisme": ["Imaginaire", "Liberté créative", "Poésie automatique", "Rêve", "Inconscient"],
    "Techniques": ["Innovations", "Science", "Technologie"],
    "Théâtre": ["Drame", "Théâtre", "Théâtre en vers"],
    "Tragique": ["funéraires", "homicide", "meurtre", "pendaison", "tragique", "victimes", "tragi-comédie", "mort"],
    "Utopie": ["Fiction spéculative", "Littérature utopique", "Socialisme utopique", "Utopie"],
    "Vengeance": ["colère", "haine", "vengeance"],
    "Vie sociale et mœurs": ["Conditions morales", "Civilisation", "Mœurs et coutumes", "Vie sociale"],
    "Voyages": ["Voyages", "Voyages imaginaires"],
    "Voyages et explorations": ["Découvertes", "Explorations", "Voyages et découvertes"],
}

# On applique la fonction de mapping, 'f_map_themes' sur la colonne "Themes"
df_books["Themes"] = df_books["Themes"].apply(f_map_themes)
df_books

,Author,Title,Year,Themes
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs",1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
1,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É...",1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
2,Victor Hugo,Œuvres complètes de Victor Hugo,1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
3,Honoré de Balzac,La Comédie humaine,1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
4,Alexandre Dumas ...,Pauline,1830,"Décadence, Décadence, Décadence, Décadence, Dé..."
...,...,...,...,...
292,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Décadence, Roman, Coutumes"
293,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Déca..."
294,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Déca..."
295,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Décadence, Roman, Coutumes"


In [31]:
# On enlève les livres qui n'ont pas de thèmes pertinents associés
df_books = df_books[df_books['Themes'] != '']
df_books.reset_index(drop=True, inplace=True)

# On retire les thèmes en double et les auteurs 
df_books['Themes'] = df_books['Themes'].apply(lambda x: ', '.join(sorted(set(x.split(', ')), key=x.split(', ').index)))
df_books = df_books[df_books['Author'].notna()]
df_books = df_books[df_books['Themes'] != '']
df_books.reset_index(drop=True, inplace=True)

df_books

/tmp/ipykernel_70065/4142290145.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Themes'] = df_books['Themes'].apply(lambda x: ', '.join(sorted(set(x.split(', ')), key=x.split(', ').index)))


,Author,Title,Year,Themes
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs",1800,Décadence
1,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É...",1800,Décadence
2,Victor Hugo,Œuvres complètes de Victor Hugo,1800,"Décadence, Coutumes"
3,Honoré de Balzac,La Comédie humaine,1800,Décadence
4,Alexandre Dumas ...,Pauline,1830,Décadence
...,...,...,...,...
219,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Roman, Coutumes"
220,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Rena..."
221,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Rena..."
222,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Roman, Coutumes"


## Sauvegarde du DataFrame avec les thèmes

Le DataFrame, enrichi avec les colonnes des thèmes, est enregistré pour permettre une utilisation ultérieure sans avoir à répéter les traitements précédents.

In [43]:
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/Data_libroguessr/mapping.csv"

with fs.open(FILE_PATH_OUT_S3, "w") as file_out:
    df_books.to_csv(file_out)

"""df_books.to_csv('Data/mapping.csv', index=False)"""

In [ ]:
FILE_PATH_IN_S3 = f"{MY_BUCKET}/Data_libroguessr/mapping.csv"

with fs.open(FILE_PATH_IN_S3, "r") as file_in:
    df_books = pd.read_csv(file_in)
    
# Après le nettoyage, le data frame prend la forme suivante :

df_books
"""df_books = pd.read_csv('Data/mapping.csv')
"""

## Poursuite du nettoyage de la base de données

Dans cette étape, nous appliquons plusieurs opérations de nettoyage pour affiner notre base de données et la rendre plus pertinente pour l'analyse :

1. **Suppression des espaces inutiles** : 
   Les espaces à la fin des noms des auteurs sont supprimés.

2. **Retrait d'une œuvre spécifique** : 
   "La Comédie humaine" de Balzac est retirée de la base, car il s'agit d'une œuvre complète et non d'un livre spécifique.

3. **Suppression des œuvres complètes** : 
   Toutes les lignes où le titre commence par des expressions comme *Œuvres complètes*, *Œuvres*, *Oeuvres*, *Poésies Complètes* ou *Ouvres* sont éliminées, car de même que pour "La Comédie humaine", on souhaite avoir des romans et non les oeuvres complètes des auteurs.

4. **Filtrage par date** : 
   Seuls les livres publiés avant 1900 sont conservés.

In [34]:
# On supprime les espaces après le nom des auteurs
df_books['Author_Cleaned'] = df_books['Author'].apply(lambda x: x.rstrip())

# Comme La Comédie humaine est l'oeuvre de Balzac est non pas un livre, on le retire de la base de données
df_books = df_books[df_books['Author_Cleaned'] != "Honoré de Balzac"]
df_books.reset_index(drop=True, inplace=True)

# Supprimer les lignes où le titre commence par 'Œuvres complètes'
df_books = df_books[~df_books['Title'].str.startswith(('Œuvres complètes', 'Œuvres', 'Oeuvres', 'Poésies Complètes', 'Ouvres'))]
df_books.reset_index(drop=True, inplace=True)

# On garde uniquement les livres écrits avant 1900
df_books = df_books[df_books['Year'] < 1900]
df_books.reset_index(drop=True, inplace=True)

#df_books = f_remove_similar_books_from_df(df_books)

df_books

,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Théophile Gautier ...,Constantinople,1834,Décadence,Théophile Gautier
2,Jean-Jacques Rousseau,Les Rêveries du Promeneur Solitaire,1762,"Société et politique, Décadence, Vie sociale e...",Jean-Jacques Rousseau
3,Edmond Rostand,Deux romanciers de Provence,1821,"Drame, Fiction historique, Décadence, Passion,...",Edmond Rostand
4,Victor Hugo,Post-scriptum de ma vie,1800,"Décadence, Coutumes",Victor Hugo
...,...,...,...,...,...
166,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Roman, Coutumes",Stendhal
167,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Rena...",Jules Verne
168,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Rena...",Jules Verne
169,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Roman, Coutumes",Stendhal


## Limitation du nombre de livres par auteur

Dans notre base de données, certains auteurs sont surreprésentés, comme Georges Sand, Jules Verne, Alexandre Dumas ou Victor Hugo, ce qui pourrait introduire des biais dans le processus de clustering. Afin de garantir une répartition plus équilibrée et représentative, nous avons décidé de limiter le nombre de livres par auteur à deux.

### Avantages de cette réduction :
1. **Équilibre dans le clustering** : Réduire le nombre de livres par auteur permet d'éviter que certains auteurs dominent l'analyse, améliorant ainsi la pertinence des clusters.
2. **Optimisation des données** : Réduire la taille des données, facilitant leur manipulation et leur exploitation tout en diminuant les besoins en ressources computationnelles.


In [35]:
# Étape 1: Compter le nombre de livres par auteur
author_counts = df_books['Author_Cleaned'].value_counts()
print(author_counts) ####### TODO on remarque que Sand a 48 livres dans le DF soit biais 

# Étape 2: Filtrer les auteurs ayant plus de 2 livres
authors_with_more_than_two_books = author_counts[author_counts > 2].index

# Étape 3: Filtrer et garder les 2 premiers livres par auteur
df_books = (
    df_books[df_books['Author_Cleaned'].isin(authors_with_more_than_two_books)]
    .sort_values(['Author_Cleaned', 'Year'])  # Trier par auteur et par année ou autre critère
    .groupby('Author_Cleaned')
    .head(10)  # Garder les 10 premiers par groupe
)
df_books.reset_index(drop=True, inplace=True)

df_books

Author_Cleaned
Jules Verne                                        34
Alexandre Dumas                                    29
Victor Hugo                                        15
Guy de Maupassant                                  15
Voltaire                                           10
Émile Zola                                          9
Alphonse Daudet                                     9
Théophile Gautier                                   7
Gustave Flaubert                                    6
Stendhal                                            5
Alexandre Dumas, Pere                               4
Edmond de Goncourt                                  3
Edmond de Goncourt, Jules de Goncourt               3
Eugène Sue                                          3
Jean de La Fontaine                                 3
Paul Scarron                                        2
Octave Mirbeau                                      2
Théophile Gautier,                                  1
Edmond Rostan

,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Alexandre Dumas,Les trois mousquetaires of 2,1830,Décadence,Alexandre Dumas
2,Alexandre Dumas,Les trois mousquetaires of 2,1830,Décadence,Alexandre Dumas
3,Alexandre Dumas ...,Gabriel Lambert,1830,Décadence,Alexandre Dumas
4,Alexandre Dumas ...,Le comte de Moret,1830,Décadence,Alexandre Dumas
...,...,...,...,...,...
97,Émile Zola ...,Le rêve,1885,"Décadence, Société et politique",Émile Zola
98,Émile Zola,Au bonheur des dames,1885,"Décadence, Société et politique",Émile Zola
99,Émile Zola,Nouveaux Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola
100,Émile Zola ...,Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola


## Nettoyage des doublons dans les livres

Dans cette étape, nous effectuons un nettoyage supplémentaire pour éliminer les doublons et corriger certains problèmes spécifiques liés aux titres des livres :

1. **Suppression des suffixes inutiles** :  
   Les titres contenant un point (.) sont tronqués pour ne conserver que la partie avant le point.

2. **Élimination des doublons** :  
   Les doublons sont supprimés sur la base de la colonne *Title*.

3. **Doublon spécifique de Victor Hugo** :  
   Un livre de Victor Hugo, *Le Rhin I*, reste identifié comme un doublon. Il est retiré manuellement.

4. **Problèmes liés aux accents** :  
   Les titres contenant des caractères problématiques, tels que `?`, sont supprimés pour éviter les erreurs dans le traitement.

5. **Suppression des suffixes en anglais** :  
   Pour certains livres, le numéro de tome est indiqué par *of* (ex. *Volume 1 of 3*). Cette partie est retirée pour normaliser les titres.

6. **Doublon spécifique d'Émile Zola** :  
   Un livre d'Émile Zola, *Nouveaux Contes à Ninon*, reste également en double. Ce doublon est supprimé manuellement.

7. **Réinitialisation des index** :  
   Après ces opérations, les index du DataFrame sont réinitialisés pour garantir une numérotation cohérente.


In [36]:
# Il reste des doublons à nettoyer dans les livres
df_books['Title'] = df_books['Title'].apply(lambda x: x.split('.')[0])
df_books = df_books.drop_duplicates(subset='Title')
# L'algorithme ne reconnaît pas qu'un livre de Victor Hugo reste un doublon. On le retire manuellement
df_books = df_books[df_books['Title'] != 'Le Rhin I']
# Les mots ayant des accents ont eu des problèmes dans la conversion des données
df_books = df_books[~df_books['Title'].str.contains('\?', regex=True)]
# Pour certains livres, le numéro du tome est indiqué en anglais par 'of'
df_books['Title'] = df_books['Title'].apply(lambda x: x.split(' of')[0])
# De même, un livre d'Emile Zola reste en double dans le dataset
df_books = df_books[df_books['Title'] != 'Nouveaux Contes à Ninon']

df_books.reset_index(drop=True, inplace=True)

df_books

<>:7: SyntaxWarning: invalid escape sequence '\?'
<>:7: SyntaxWarning: invalid escape sequence '\?'
/tmp/ipykernel_70065/3527136753.py:7: SyntaxWarning: invalid escape sequence '\?'
  df_books = df_books[~df_books['Title'].str.contains('\?', regex=True)]


,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Alexandre Dumas,Les trois mousquetaires,1830,Décadence,Alexandre Dumas
2,Alexandre Dumas ...,Gabriel Lambert,1830,Décadence,Alexandre Dumas
3,Alexandre Dumas ...,Le comte de Moret,1830,Décadence,Alexandre Dumas
4,Alexandre Dumas ...,La tulipe noire,1830,Décadence,Alexandre Dumas
...,...,...,...,...,...
83,Émile Zola ...,La curée,1885,"Décadence, Société et politique",Émile Zola
84,Émile Zola ...,Le rêve,1885,"Décadence, Société et politique",Émile Zola
85,Émile Zola,Au bonheur des dames,1885,"Décadence, Société et politique",Émile Zola
86,Émile Zola ...,Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola


In [48]:
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/Data_libroguessr/final_list.csv"

with fs.open(FILE_PATH_OUT_S3, "w") as file_out:
    df_books.to_csv(file_out, index = False)


"df_books.to_csv('Data/final_list.csv', index=False)"